In [ ]:
import os
import time
import shutil
from pathlib import Path
import sqlite3

def merge_chroma_sqlite_with_copy(
    src_db_path: str,
    dst_db_path: str,
    tables: list[str],
    copy_suffix: str = "_temp_merge_copy", # 중간 병합을 위한 고유 접미사
    max_retries: int = 5,
    retry_delay: float = 5.0
):
    """
    소스(src) Chroma SQLite DB의 데이터를 대상(dst) Chroma SQLite DB의 복사본에 병합합니다.
    원본 대상 DB는 보존하고, 대상 DB의 복사본 파일에 src DB의 테이블 데이터를 이어붙입니다.
    'collections' 테이블은 스킵하며, INSERT OR IGNORE로 중복 엔트리를 무시합니다.

    Args:
        src_db_path: 소스 DB 파일 경로 (데이터를 가져올 DB)
        dst_db_path: 대상 DB 파일 경로 (데이터가 병합될 복사본의 원본이 될 DB)
        tables: 병합할 테이블 이름 리스트 (예: ['segments', 'embeddings', 'embedding_metadata'])
        copy_suffix: 대상 복사본 파일명에 붙일 접미사
        max_retries: DB 잠김 해제 재시도 횟수
        retry_delay: 재시도 대기 시간(초)
    """
    src = Path(src_db_path)
    dst = Path(dst_db_path)

    if not src.exists():
        raise FileNotFoundError(f"소스 DB를 찾을 수 없습니다: {src}")
    if not dst.exists():
        raise FileNotFoundError(f"대상 DB를 찾을 수 없습니다: {dst}")

    # 1) 대상 DB의 복사본 생성 (이 복사본에 src DB 데이터를 병합)
    # 복사본의 이름은 원본 대상 DB 이름에 접미사를 붙여 생성합니다.
    dst_copy = dst.with_name(dst.stem + copy_suffix + dst.suffix)
    
    # 원본 대상 DB가 이미 복사본이라면, 기존 복사본을 삭제하고 새로 복사합니다.
    # 이렇게 해야 이전 병합의 결과가 다음 병합의 대상이 될 수 있습니다.
    if dst_copy.exists():
        os.remove(dst_copy)
        print(f"기존 임시 복사본 삭제: {dst_copy}")

    shutil.copy2(dst, dst_copy)
    print(f"'{dst.name}'의 임시 복사본 생성: '{dst_copy.name}'")

    # 2) 복사본에 병합 수행
    attempt = 0
    while True:
        try:
            # 복사본 DB에 연결합니다.
            conn = sqlite3.connect(str(dst_copy), timeout=30, uri=True)
            conn.execute("PRAGMA journal_mode=WAL;") # WAL 모드 활성화로 동시성 및 복구력 향상
            conn.execute("PRAGMA busy_timeout=30000;") # 잠김 시 30초 대기
            cur = conn.cursor()

            # 소스 DB를 읽기 전용으로 ATTACH합니다.
            # 'file:...' URI 형식은 Windows 경로에서도 작동하며, mode=ro는 읽기 전용을 의미합니다.
            attach_str = f"file:{src}?mode=ro"
            cur.execute(f"ATTACH DATABASE '{attach_str}' AS src KEY ''; ")
            cur.execute("BEGIN;") # 트랜잭션 시작

            for tbl in tables:
                # 'collections' 테이블은 ChromaDB의 메타데이터를 관리하므로 병합 시 스킵합니다.
                # 각 DB의 collection ID가 다를 수 있어 직접 병합하면 충돌 가능성이 높습니다.
                if tbl == 'collections':
                    print(f"'{tbl}' 테이블은 스킵합니다.")
                    continue
                
                # INSERT OR IGNORE: 중복되는 primary key가 있으면 해당 행을 무시하고 넘어갑니다.
                # 이는 ChromaDB의 ID 중복을 방지하는 데 유용합니다.
                cur.execute(
                    f"INSERT OR IGNORE INTO {tbl} SELECT * FROM src.{tbl};"
                )
                print(f"'{tbl}' 테이블에 {cur.rowcount}개의 행을 병합했습니다.")

            cur.execute("COMMIT;") # 트랜잭션 커밋
            cur.execute("DETACH DATABASE src;") # 소스 DB 연결 해제
            conn.close() # DB 연결 닫기
            print(f"임시 복사본 '{dst_copy.name}'에 병합 완료.")
            break # 성공했으므로 루프 종료

        except sqlite3.OperationalError as e:
            # DB 잠김 오류 발생 시 재시도
            if 'locked' in str(e).lower() and attempt < max_retries:
                attempt += 1
                print(f"DB가 잠겨 있습니다. {attempt}/{max_retries} 재시도 중... {retry_delay}초 후 다시 시도합니다.")
                time.sleep(retry_delay)
                continue
            raise # 재시도 횟수를 초과했거나 다른 sqlite3 오류인 경우 예외 발생

if __name__ == '__main__':
    # --- 설정 ---
    # 당신의 실제 ChromaDB 폴더들이 위치한 상위 경로로 변경해주세요.
    # 예: C:/Users/mojih/OneDrive/바탕 화면/Project4/SKN13-4th-4Team/ai_influencer_agent/
    BASE_DB_DIR = r"C:\Users\mojih\OneDrive\바탕 화면\Project4\SKN13-4th-4Team\crawling" 

    # 병합할 ChromaDB 폴더 이름 리스트 (순서대로 병합됩니다)
    # 각 폴더 안에 'chroma.sqlite3' 파일이 있어야 합니다.
    DB_FOLDERS = [
        "chroma_pubmed_abstract_only",
        "chroma_rehab_db",
        "chroma_rehab_db_openai",
        "chroma_rehab_db_pmc_openai",
        "pilates_expert_blog_db"
    ]

    # ChromaDB의 주요 테이블 목록
    # 'collections' 테이블은 ID 충돌 가능성 때문에 병합 로직에서 스킵됩니다.
    TABLES_TO_MERGE = ['segments', 'embeddings', 'embedding_metadata']

    # 최종 병합된 DB가 저장될 이름 (새로운 파일로 생성됩니다)
    FINAL_MERGED_DB_NAME = "merged_rehabilitation_chroma.sqlite3"
    
    print("--- ChromaDB 파일 병합 시작 ---")

    # 첫 번째 DB를 초기 대상 DB로 설정합니다.
    # 이 파일은 병합 과정에서 임시 복사본의 원본으로 사용됩니다.
    initial_dst_db_path = Path(BASE_DB_DIR) / DB_FOLDERS[0] / "chroma.sqlite3"
    if not initial_dst_db_path.exists():
        raise FileNotFoundError(f"초기 대상 DB를 찾을 수 없습니다: {initial_dst_db_path}")
    
    # 병합 결과를 저장할 현재 대상 DB 경로를 초기화합니다.
    # 첫 번째 DB 파일 자체가 첫 번째 병합의 대상이 됩니다 (이것의 복사본이 생성됨).
    current_merged_db_path = initial_dst_db_path 

    # 나머지 DB들을 순차적으로 병합합니다.
    # 각 반복에서 current_merged_db_path는 이전 병합의 결과(임시 복사본)를 가리키게 됩니다.
    for i in range(1, len(DB_FOLDERS)):
        src_folder_name = DB_FOLDERS[i]
        src_db_path = Path(BASE_DB_DIR) / src_folder_name / "chroma.sqlite3"
        
        print(f"\n--- '{src_folder_name}' DB를 현재 병합 결과에 병합 중 ---")
        print(f"소스 DB: '{src_db_path.name}'")
        print(f"대상 DB (원본): '{current_merged_db_path.name}'")

        # merge_chroma_sqlite_with_copy 함수는 dst_db_path의 복사본에 src_db_path를 병합합니다.
        # 반환값은 없지만, dst_copy 파일이 생성됩니다.
        merge_chroma_sqlite_with_copy(
            src_db_path=str(src_db_path),
            dst_db_path=str(current_merged_db_path),
            tables=TABLES_TO_MERGE,
            # 다음 병합의 대상이 될 임시 파일의 이름을 지정합니다.
            # 예: chroma_pubmed_abstract_only_temp_merge_copy.sqlite3
            #     chroma_pubmed_abstract_only_temp_merge_copy_temp_merge_copy.sqlite3
            # 이렇게 이름이 길어지는 것을 방지하기 위해, 매번 새로운 임시 파일 이름을 생성합니다.
            copy_suffix=f"_merged_{i}" 
        )
        
        # 다음 병합을 위해, 방금 생성된 임시 복사본을 새로운 대상 DB로 설정합니다.
        # 이 임시 복사본이 다음 병합의 'dst' 역할을 하게 됩니다.
        current_merged_db_path = current_merged_db_path.with_name(
            current_merged_db_path.stem + f"_merged_{i}" + current_merged_db_path.suffix
        )
        print(f"다음 병합의 대상 DB로 설정: '{current_merged_db_path.name}'")

    # 모든 병합이 완료된 후, 최종 병합된 DB 파일의 이름을 변경합니다.
    final_db_path = Path(BASE_DB_DIR) / FINAL_MERGED_DB_NAME
    
    # 최종 결과 파일이 이미 존재하면 삭제합니다.
    if final_db_path.exists():
        os.remove(final_db_path)
        print(f"기존 최종 병합 DB 삭제: {final_db_path}")

    # 마지막으로 생성된 임시 병합 파일을 최종 이름으로 변경합니다.
    # 만약 DB_FOLDERS가 하나만 있다면, initial_dst_db_path를 복사합니다.
    if len(DB_FOLDERS) == 1:
        shutil.copy2(initial_dst_db_path, final_db_path)
        print(f"단일 DB를 최종 DB로 복사: {final_db_path.name}")
    else:
        shutil.move(current_merged_db_path, final_db_path)
        print(f"최종 병합된 DB '{current_merged_db_path.name}'를 '{final_db_path.name}'(으)로 이름 변경 완료.")

    print("\n--- 모든 ChromaDB 파일 병합 완료 ---")
    print(f"최종 병합된 DB 파일: {final_db_path}")



--- ChromaDB 파일 병합 시작 ---


FileNotFoundError: 초기 대상 DB를 찾을 수 없습니다: C:\Users\mojih\OneDrive\바탕 화면\Project4\SKN13-4th-4Team\ai_influencer_agent\chroma_pubmed_abstract_only\chroma.sqlite3